<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoenc_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow==2.1.0-rc0

In [2]:
pip install keras==2.3.1

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "aba8dc1f085221111d925003fe5a88ed" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

train_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_identity.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_transaction.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
trn=pd.read_csv('train_transaction.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')
trn=trn.drop(['isFraud'],1)

In [6]:
ls=list(trn.filter(regex='V'))
trn=trn.drop(ls,1)
tst=tst.drop(ls,1)
trn.shape

(590540, 54)

In [0]:
import gc
fnl=pd.concat([trn,tst],0)
fnl=fnl.reset_index(drop=True)
del([trn,tst])
gc.collect()
def ext(fnl):

  fnl['P_1']=fnl['P_emaildomain'].fillna('nan').str.split('.').str[0]
  fnl['P_2']=fnl['P_emaildomain'].fillna('nan').str.split('.').str[1]


  fnl['R_1']=fnl['R_emaildomain'].fillna('nan').str.split('.').str[0]
  fnl['R_2']=fnl['R_emaildomain'].fillna('nan').str.split('.').str[1]
  return fnl
fnl=ext(fnl)

In [0]:
fnl=fnl.drop(['P_emaildomain','R_emaildomain','TransactionDT','TransactionID'],1)


In [0]:
def handle_disc(df,cols,num):
  for col in cols:
    par=min(df[col].nunique(),num)
    unq = df[col].value_counts().head(par)
    dum=pd.get_dummies(df[col])[unq.index]
    ls=[]
    for i in dum.columns:
      ls.append(str(col)+str(i))
    dum.columns=ls
    df=pd.concat([df,dum],1)

  df=df.drop(cols,1)
  sp=df.shape[1]
  df.columns=list(range(sp))
  return df
def handle_cont(df,cols):
  for col in cols:
    df[col]=df[col].fillna(df[col].mean())
    mn=df[col].mean()
    std=df[col].std()
    df[col]=(df[col]-mn)/(std+0.1)
    df[col]=np.log10(df[col]+1-min(0,min(df[col])))
  return df[cols]
disc=['ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','P_1','P_2','R_1','R_2','M1','M2','M3','M4','M5','M6','M7','M8','M9']
cont=[i for i in fnl.columns if i not in disc]
fnl_cont=handle_cont(fnl,cont)
fnl_cont=fnl_cont.replace([np.inf,-np.inf],0)
fnl_disc=handle_disc(fnl,disc,140)
fnl=pd.concat([fnl_cont,fnl_disc],1).reset_index(drop=True)



In [10]:
import gc
import os
gc.collect()
import numpy as np
import keras 
from math import ceil
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout,BatchNormalization
from keras.layers import Add
import keras.backend as K
class DAESequence(Sequence):
    def __init__(self,df,batch_size):
        self.batch_size=batch_size
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df.values
        self.idx=[]
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min((idx+1)*self.batch_size,self.len_data)
        idx=idx*self.batch_size
        size=last-idx
        output_x=self.data[idx:last]
        length=last-idx
        rnd=np.random.randint(0,self.len_data-size,size)
        rnd[rnd>idx]+=size
        cols=np.random.randint(0,self.columns,int(self.columns*0.1))
        noise_x=output_x.copy()
        noise_x[:,cols]=self.data[rnd[:,None],cols]
        return noise_x,output_x
from keras import backend as K


class WarmUpLearningRateScheduler(keras.callbacks.Callback):
    """Warmup learning rate scheduler
    """

    def __init__(self, warmup_batches, init_lr, verbose=0):
        """Constructor for warmup learning rate scheduler

        Arguments:
            warmup_batches {int} -- Number of batch for warmup.
            init_lr {float} -- Learning rate after warmup.

        Keyword Arguments:
            verbose {int} -- 0: quiet, 1: update messages. (default: {0})
        """

        super(WarmUpLearningRateScheduler, self).__init__()
        self.warmup_batches = warmup_batches
        self.init_lr = init_lr
        self.verbose = verbose
        self.batch_count = 0
        self.learning_rates = []

    def on_batch_end(self, batch, logs=None):
        self.batch_count = self.batch_count + 1
        lr = K.get_value(self.model.optimizer.lr)
        self.learning_rates.append(lr)

    def on_batch_begin(self, batch, logs=None):
        if self.batch_count <= self.warmup_batches:
            lr = self.batch_count*self.init_lr/self.warmup_batches
            K.set_value(self.model.optimizer.lr, lr)
            if self.verbose > 0:
                print('\nBatch %05d: WarmUpLearningRateScheduler setting learning '
                      'rate to %s.' % (self.batch_count + 1, lr))
warm_up_lr = WarmUpLearningRateScheduler(400, init_lr=0.005)
from keras.models import Sequential
from keras.callbacks.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.layers import BatchNormalization
#fnl=pd.concat([fnl_trn,fnl_tst],0).select_dtypes(exclude='uint8')
batch_size=1024
checkpoint_path = "training_1.hdf5" 
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback =ModelCheckpoint(checkpoint_path, 
monitor='loss',save_best_only=True,save_weights_only=True,verbose=1)
len_input_columns=fnl.shape[1]
model_dae = Sequential()
model_dae.add(Dense(units=len_input_columns,input_shape=(len_input_columns,), activation='relu', dtype='float32', name='Input'))
model_dae.add(Dense(units=int(512), activation='relu', dtype='float32',name='Hidden_1'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(256), activation='relu', dtype='float32',name='Hidden_2'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(512), activation='relu', dtype='float32',name='Hidden_3'))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=len_input_columns, activation='linear', dtype='float32', name='Output'))
model_dae.compile(loss='mean_squared_error',optimizer='adam')
hist=model_dae.fit_generator(DAESequence(fnl, batch_size=2048),steps_per_epoch=int(ceil(fnl.shape[0]/batch_size)),epochs=10,workers=2,callbacks=[warm_up_lr])

Using TensorFlow backend.


Epoch 1/10
1072/1072 [==============================] - 38s 35ms/step - loss: 0.0236
Epoch 2/10
1072/1072 [==============================] - 35s 33ms/step - loss: 0.0037
Epoch 3/10
1072/1072 [==============================] - 36s 33ms/step - loss: 0.0026
Epoch 4/10
1072/1072 [==============================] - 36s 33ms/step - loss: 0.0021
Epoch 5/10
1072/1072 [==============================] - 36s 33ms/step - loss: 0.0018
Epoch 6/10
1072/1072 [==============================] - 35s 33ms/step - loss: 0.0016
Epoch 7/10
1072/1072 [==============================] - 36s 33ms/step - loss: 0.0015
Epoch 8/10
1072/1072 [==============================] - 35s 33ms/step - loss: 0.0014
Epoch 9/10
1072/1072 [==============================] - 35s 33ms/step - loss: 0.0014
Epoch 10/10
1072/1072 [==============================] - 35s 33ms/step - loss: 0.0013


In [0]:
path = F"/content/gdrive/My Drive/autoenc_final.hdf5" 
model_dae.save_weights(path)

In [12]:

from tqdm import tqdm
your_new_df=pd.DataFrame()
for i in tqdm([3]):
    intermediate_layer_model = Model(inputs=model_dae.input, outputs=model_dae.layers[i].output)
    z = pd.DataFrame(intermediate_layer_model.predict(fnl))
    columns_names = ['Hidden_'+str(i)+'_'+str(l) for l in range(0, z.shape[1])]
    z.columns=columns_names
    your_new_df=pd.concat([your_new_df,z],1)

100%|██████████| 1/1 [00:54<00:00, 54.48s/it]


In [13]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
your_new_df=reduce_mem_usage(your_new_df)

Memory usage of dataframe is 1071.51 MB
Memory usage after optimization is: 535.76 MB
Decreased by 50.0%


In [0]:
trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
your_new_df=your_new_df.loc[:trn.shape[0]]
your_new_df['isFraud']=trn['isFraud']
fnl=pd.DataFrame()
cols=[]
val=[]

In [0]:
your_new_df=your_new_df.dropna()
your_new_df=your_new_df.replace([np.inf,-np.inf],0)

In [0]:
ls=list(your_new_df)
ls.remove('isFraud')

In [17]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
trn=[]
val=[]
cols=[]
for col in tqdm(ls):
    for i in range(50000,590540,40000):
        fit=your_new_df.loc[0:i]
        pdct=your_new_df.loc[i:i+40000]
        mod=lgb.LGBMClassifier(max_bins= 37,max_depth= 40,min_data_in_leaf= 110,min_gain_to_split= 1,num_leaves=48)
        mod.fit(fit[col].values.reshape(-1,1),fit['isFraud'])
        val.append(roc_auc_score(pdct['isFraud'],mod.predict_proba(pdct[col].values.reshape(-1,1))[:,1]))
        trn.append(roc_auc_score(fit['isFraud'],mod.predict_proba(fit[col].values.reshape(-1,1))[:,1]))
        cols.append(col)

100%|██████████| 256/256 [52:29<00:00, 12.30s/it]



In [18]:
sdo=pd.DataFrame()
sdo['cols']=cols
sdo['validation']=val
sdo['train']=trn
sdo['validation']=sdo.groupby(['cols'])['validation'].transform('mean')
sdo['train']=sdo.groupby(['cols'])['train'].transform('mean')
sdo=sdo[['validation','train','cols']]
sdo=sdo.drop_duplicates(keep='last')
sdo=sdo.sort_values(by='validation')
sdo.tail(100)['cols'].values

array(['Hidden_3_84', 'Hidden_3_158', 'Hidden_3_125', 'Hidden_3_82',
       'Hidden_3_214', 'Hidden_3_25', 'Hidden_3_218', 'Hidden_3_23',
       'Hidden_3_176', 'Hidden_3_229', 'Hidden_3_80', 'Hidden_3_231',
       'Hidden_3_106', 'Hidden_3_28', 'Hidden_3_44', 'Hidden_3_92',
       'Hidden_3_230', 'Hidden_3_72', 'Hidden_3_20', 'Hidden_3_69',
       'Hidden_3_13', 'Hidden_3_221', 'Hidden_3_37', 'Hidden_3_24',
       'Hidden_3_144', 'Hidden_3_254', 'Hidden_3_208', 'Hidden_3_71',
       'Hidden_3_193', 'Hidden_3_184', 'Hidden_3_240', 'Hidden_3_3',
       'Hidden_3_96', 'Hidden_3_188', 'Hidden_3_198', 'Hidden_3_234',
       'Hidden_3_52', 'Hidden_3_255', 'Hidden_3_137', 'Hidden_3_6',
       'Hidden_3_17', 'Hidden_3_126', 'Hidden_3_19', 'Hidden_3_48',
       'Hidden_3_145', 'Hidden_3_135', 'Hidden_3_148', 'Hidden_3_139',
       'Hidden_3_91', 'Hidden_3_209', 'Hidden_3_146', 'Hidden_3_43',
       'Hidden_3_120', 'Hidden_3_99', 'Hidden_3_26', 'Hidden_3_155',
       'Hidden_3_62', 'Hidden_3_33

array(['Hidden_3_84', 'Hidden_3_158', 'Hidden_3_125', 'Hidden_3_82',
       'Hidden_3_214', 'Hidden_3_25', 'Hidden_3_218', 'Hidden_3_23',
       'Hidden_3_176', 'Hidden_3_229', 'Hidden_3_80', 'Hidden_3_231',
       'Hidden_3_106', 'Hidden_3_28', 'Hidden_3_44', 'Hidden_3_92',
       'Hidden_3_230', 'Hidden_3_72', 'Hidden_3_20', 'Hidden_3_69',
       'Hidden_3_13', 'Hidden_3_221', 'Hidden_3_37', 'Hidden_3_24',
       'Hidden_3_144', 'Hidden_3_254', 'Hidden_3_208', 'Hidden_3_71',
       'Hidden_3_193', 'Hidden_3_184', 'Hidden_3_240', 'Hidden_3_3',
       'Hidden_3_96', 'Hidden_3_188', 'Hidden_3_198', 'Hidden_3_234',
       'Hidden_3_52', 'Hidden_3_255', 'Hidden_3_137', 'Hidden_3_6',
       'Hidden_3_17', 'Hidden_3_126', 'Hidden_3_19', 'Hidden_3_48',
       'Hidden_3_145', 'Hidden_3_135', 'Hidden_3_148', 'Hidden_3_139',
       'Hidden_3_91', 'Hidden_3_209', 'Hidden_3_146', 'Hidden_3_43',
       'Hidden_3_120', 'Hidden_3_99', 'Hidden_3_26', 'Hidden_3_155',
       'Hidden_3_62', 'Hidden_3_33

In [0]:
path = F"/content/gdrive/My Drive/extra__1.pkl" 
your_new_df.to_pickle(path)

27

KeyboardInterrupt: ignored